In [52]:
# Installations

%pip install --upgrade pip -q
%pip install python-dotenv -q

%pip install langchain langchain_openai langgraph duckduckgo-search -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [53]:
from dotenv import dotenv_values

environment_secrets = dotenv_values("../.env")

In [54]:

from langchain_openai import ChatOpenAI
from pydantic.v1 import SecretStr

openai_api_key = environment_secrets["OPENAI_API_KEY"]

assert openai_api_key != None, "OpenAI Api Key not found!"

openai_client = ChatOpenAI(
    api_key=SecretStr(openai_api_key)
)

In [55]:
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain_core.tools import tool

tools = [
    DuckDuckGoSearchRun()
]

openai_client = openai_client.bind_tools(tools)

In [56]:
from langgraph.graph import END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, SystemMessage
from uuid import uuid4

def converse(state: MessagesState) -> MessagesState:
    return {
        "messages": [
            openai_client.invoke(state["messages"])
		]
	}

def decide(state: MessagesState) -> str:
	latest_message = state["messages"][-1]

	print(latest_message)

	return (
		"tools" if 
		(
			tool_calls := getattr(latest_message, "tool_calls", None)
		) != None and 
		len(tool_calls) >= 1 else 
		END
	)

def main():
	agent_flow = StateGraph(MessagesState)

	agent_flow.add_node("converse", converse)
	agent_flow.add_node("decide", decide)
	agent_flow.add_node("tools", ToolNode(tools))

	agent_flow.set_entry_point("converse")
	agent_flow.add_conditional_edges(
		"converse",
		decide
	)
	agent_flow.add_edge("tools", "converse")

	agent = agent_flow.compile()

	thread_id = uuid4().int

	response = agent.invoke(
		{
			"messages": [
				SystemMessage(content="If you do not know the answer to a question, try calling a tool. If you called a tool, include quotes from your sources to show proof."),
				HumanMessage(content="What is the temperature right now in Silang, Cavite, Philippines?")
			]
		},
		config={"configurable": {"conversation_id": thread_id}}
	)

	print(response)

In [57]:
from langchain_core.tracers.context import tracing_v2_enabled
from langsmith import Client

from uuid import uuid4

langsmith_client = Client(api_key=environment_secrets["LANGSMITH_API_KEY"])

if __name__ == "__main__":
	with tracing_v2_enabled(project_name=f"Suds Ai - LangGraph - {uuid4().hex[0:8]}", client=langsmith_client):
		main()

content='' additional_kwargs={'tool_calls': [{'id': 'call_PlYQqLzEPE2JdWXywGuehrHi', 'function': {'arguments': '{"query":"current temperature in Silang, Cavite, Philippines"}', 'name': 'duckduckgo_search'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 122, 'total_tokens': 148}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run-143e2304-9c2d-46ba-ac2e-43475442b306-0' tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'current temperature in Silang, Cavite, Philippines'}, 'id': 'call_PlYQqLzEPE2JdWXywGuehrHi', 'type': 'tool_call'}] usage_metadata={'input_tokens': 122, 'output_tokens': 26, 'total_tokens': 148}
content='The current temperature in Silang, Cavite, Philippines is 26°C (78.8°F), with a felt air temperature of 30°C (86°F) due to the relative humidity.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'comple